In [ ]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

# Load the data
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
application_df = pd.read_csv(url)

# Display the first few rows
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the EIN and NAME columns
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [ ]:
# Combine rare application types into "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
cutoff = 500
application_types_to_replace = application_type_counts[application_type_counts < cutoff].index.tolist()
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(application_types_to_replace, 'Other')

In [ ]:
# Combine rare classifications into "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
cutoff = 1000
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, 'Other')

In [ ]:
# Convert categorical data to numeric with pd.get_dummies
application_df = pd.get_dummies(application_df)

In [ ]:
# Define features and target
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_df['IS_SUCCESSFUL'].values

In [ ]:
# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define the learning rate reduction and early stopping callbacks
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                 patience=3,
                                 verbose=1,
                                 factor=0.5,
                                 min_lr=0.00001)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               verbose=1,
                               restore_best_weights=True)

In [ ]:
# Define a Hypermodel
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu',
                    kernel_regularizer=l2(hp.Float('l2', 0.0001, 0.01, sampling='log')),
                    input_dim=X_train.shape[1]))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)))

    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dropout_{i}', 0.2, 0.5, step=0.1)))

    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:

# Instantiate a Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=2,
    directory='my_dir',
    project_name='hyperparameter_tuning'
)

# Perform the search
tuner.search(X_train, y_train,
             epochs=150,
             validation_data=(X_test, y_test),
             callbacks=[lr_reduction, early_stopping])

Trial 20 Complete [00h 12m 14s]
val_accuracy: 0.7287901043891907

Best val_accuracy So Far: 0.7303935885429382
Total elapsed time: 04h 01m 56s


In [ ]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Evaluate the best model
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Best Model Loss: {loss}, Best Model Accuracy: {accuracy}')

215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7302 - loss: 0.5588
Best Model Loss: 0.5601128935813904, Best Model Accuracy: 0.7304664850234985


In [ ]:

# Create the LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
# Define the optimized model
nn_optimized = tf.keras.Sequential()

In [ ]:

# First hidden layer with Batch Normalization and L2 regularization
nn_optimized.add(Dense(units=128, activation='relu', kernel_regularizer=l2(0.01), input_dim=X_train.shape[1]))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.5))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

# Second hidden layer with Batch Normalization and L2 regularization
nn_optimized.add(Dense(units=64, activation='relu', kernel_regularizer=l2(0.01)))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.5))

In [ ]:
# Third hidden layer with Batch Normalization and L2 regularization
nn_optimized.add(Dense(units=32, activation='relu', kernel_regularizer=l2(0.01)))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.5))

In [ ]:
# Fourth hidden layer with Batch Normalization and L2 regularization
nn_optimized.add(Dense(units=16, activation='relu', kernel_regularizer=l2(0.01)))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.5))

In [ ]:
# Fifth hidden layer with Batch Normalization and L2 regularization
nn_optimized.add(Dense(units=8, activation='relu', kernel_regularizer=l2(0.01)))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.5))

In [ ]:
# Output layer
nn_optimized.add(Dense(units=1, activation='sigmoid'))

In [ ]:
# Compile the optimized model
nn_optimized.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the optimized model with callbacks
history = nn_optimized.fit(X_train, y_train,
                           epochs=150,
                           batch_size=32,
                           validation_data=(X_test, y_test),
                           callbacks=[lr_reduction, early_stopping])


Epoch 1/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.5204 - loss: 2.6283 - val_accuracy: 0.6790 - val_loss: 1.4727 - learning_rate: 0.0010
Epoch 2/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.6325 - loss: 1.2887 - val_accuracy: 0.7160 - val_loss: 0.8211 - learning_rate: 0.0010
Epoch 3/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7046 - loss: 0.7849 - val_accuracy: 0.7262 - val_loss: 0.6493 - learning_rate: 0.0010
Epoch 4/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7113 - loss: 0.6602 - val_accuracy: 0.7125 - val_loss: 0.6274 - learning_rate: 0.0010
Epoch 5/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7134 - loss: 0.6409 - val_accuracy: 0.7203 - val_loss: 0.6198 - learning_rate: 0.0010
Epoch 6/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7123 - loss: 0.6349 - val_accuracy: 0.7206 - val_loss: 0.6127 - learning_rate: 0.0010
Epoch 7/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7102 - loss: 0

In [ ]:
# Evaluate the optimized model
loss, accuracy = nn_optimized.evaluate(X_test, y_test)
print(f"Optimized Model Loss: {loss}, Optimized Model Accuracy: {accuracy}")


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7299 - loss: 0.5602
Optimized Model Loss: 0.5612517595291138, Optimized Model Accuracy: 0.7295918464660645


In [ ]:
# Save the optimized model to an HDF5 file
nn_optimized.save('AlphabetSoupCharity_Optimization.h5')
